Note: The section title is same as the paper [How to build your own ASP-based system?!](https://www.cambridge.org/core/journals/theory-and-practice-of-logic-programming/article/how-to-build-your-own-aspbased-system/EF826D6B5BC87BE640227CE3D65FFE18)

### 5.1 A gentle introduction

In [1]:
%%file chemistry.lp

a(1).
#program acid(k).
b(k).
c(X,k) :- a(X).
#program base.
a(2).

Overwriting chemistry.lp


In [2]:
from clingo.control import Control

ctl = Control()
ctl.load("chemistry.lp")
ctl.ground([("base", [])])
ctl.solve(on_model=print)

a(1) a(2)


SolveResult(1)

In [3]:
from clingo.symbol import Number
from clingo.control import Control
ctl = Control()
ctl.load("chemistry.lp")
ctl.ground([("acid",[Number (42)])])
ctl.solve(on_model=print)

b(42)


SolveResult(1)

### 5.2 Branch-and-bound-based optimization

In [4]:
%%file tohB.lp

% initial situation
on(D,P,0) :- init_on(D,P).

% check goal situation
ngoal(T) :- on(D,P,T), not goal_on(D,P).
:- ngoal(n).

% state transition and state constraints
1 { move(D,P,T) : disk(D), peg(P) } 1 :- ngoal(T-1), T<=n.

move(D,T)        :- move(D,P,T).
on(D,P,T)        :- move(D,P,T).
on(D,P,T)        :- on(D,P,T-1), not move(D,T), T<=n.
blocked(D-1,P,T) :- on(D,P,T-1).
blocked(D-1,P,T) :- blocked(D,P,T), disk(D).

:- move(D,P,T), blocked(D-1,P,T).
:- move(D,T), on(D,P,T-1), blocked(D,P,T).
:- disk(D), not 1 { on(D,P,T) } 1, T=1..n.

#show move/3.

_minimize(1,T) :- ngoal(T).

Overwriting tohB.lp


In [5]:
%%file tohI.lp

peg(a;b;c).
disk(1..4).
init_on(1..4,a).
goal_on(1..4,c).

Overwriting tohI.lp


In [6]:
%%file opt.py

import sys
from clingo.symbol import Number, SymbolType
from clingo.application import Application, clingo_main


class OptExampleApp(Application):

    program_name = 'opt-example'
    version = '1.0'

    def __init__(self):
        self._bound = None

    def _on_model(self, model):
        self._bound = 0
        for atom in model.symbols(atoms=True):
            if atom.match('_minimize', 2) and atom.arguments[0].type \
                is SymbolType.Number:
                self._bound += atom.arguments[0].number

    def main(self, ctl, files):
        if not files:
            files = ['-']
        for f in files:
            ctl.load(f)
        ctl.add('bound', ['b'],
                ':- #sum{V,I : _minimize (V,I)} >= b.')
        ctl.ground([('base', [])])

        while ctl.solve(on_model=self._on_model).satisfiable:
            print('Found new bound : {} '.format(self._bound))
            ctl.ground([('bound', [Number(self._bound)])])

        if self._bound is not None:
            print('Optimum found')

clingo_main(OptExampleApp(), sys.argv[1:])    

Overwriting opt.py


In [7]:
!python opt.py tohB.lp tohI.lp -c n=17

opt-example version 1.0
Reading from tohB.lp ...
Solving...
Answer: 1
move(3,c,2) move(4,b,1) move(4,a,3) move(4,c,4) move(2,b,5) move(4,a,6) move(3,b,7) move(4,c,8) move(4,b,9) move(1,c,10) move(4,c,11) move(3,a,12) move(4,a,13) move(2,c,14) move(4,b,15) move(3,c,16) move(4,c,17)
Solving...
Answer: 1
move(3,c,2) move(4,b,1) move(4,c,3) move(2,b,4) move(4,a,5) move(3,b,6) move(4,c,7) move(4,b,8) move(1,c,9) move(4,c,10) move(3,a,11) move(4,a,12) move(2,c,13) move(4,b,14) move(3,c,15) move(4,c,16)
Solving...
Answer: 1
move(3,c,2) move(4,b,1) move(4,c,3) move(2,b,4) move(4,a,5) move(3,b,6) move(4,b,7) move(1,c,8) move(4,c,9) move(3,a,10) move(4,a,11) move(2,c,12) move(4,b,13) move(3,c,14) move(4,c,15)
Solving...
UNSATISFIABLE

Models       : 3
Calls        : 4
Time         : 0.018s (Solving: 0.01s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.016s
Found new bound : 17 
Found new bound : 16 
Found new bound : 15 
Optimum found


### 5.3 Branch-and-bound-based optimization

In [8]:
%%file tohE.lp

#program base.
on(D,P,0) :- init_on(D,P).

#program check(t).
:- goal_on(D,P), not on(D,P,t), query(t).


#program step(t).
1 { move(D,P,t) : disk(D), peg(P) } 1.

move(D,t)        :- move(D,P,t).
on(D,P,t)        :- move(D,P,t).
on(D,P,t)        :- on(D,P,t-1), not move(D,t).
blocked(D-1,P,t) :- on(D,P,t-1).
blocked(D-1,P,t) :- blocked(D,P,t), disk(D).

:- move(D,P,t), blocked(D-1,P,t).
:- move(D,t), on(D,P,t-1), blocked(D,P,t).
:- disk(D), not 1 { on(D,P,t) } 1.

#show move/3.

Overwriting tohE.lp


In [9]:
%%file inc.py

'''
Example implementing an iclingo-like application.
'''

import sys
from typing import cast, Any, Callable, Optional, Sequence

from clingo.application import clingo_main, Application, ApplicationOptions
from clingo.control import Control
from clingo.solving import SolveResult
from clingo.symbol import Function, Number


class IncConfig:
    '''
    Configuration object for incremental solving.
    '''
    imin: int
    imax: Optional[int]
    istop: str

    def __init__(self):
        self.imin = 1
        self.imax = None
        self.istop = "SAT"


def parse_int(conf: Any,
              attr: str,
              min_value: Optional[int] = None,
              optional: bool = False) -> Callable[[str], bool]:
    '''
    Returns a parser for integers.

    The parser stores its result in the `attr` attribute (given as string) of
    the `conf` object. The parser can be configured to only accept integers
    having a minimum value and also to treat value `"none"` as `None`.
    '''
    def parse(sval: str) -> bool:
        if optional and sval == "none":
            value = None
        else:
            value = int(sval)
            if min_value is not None and value < min_value:
                raise RuntimeError("value too small")
        setattr(conf, attr, value)
        return True
    return parse


def parse_stop(conf: Any, attr: str) -> Callable[[str], bool]:
    '''
    Returns a parser for `istop` values.
    '''
    def parse(sval: str) -> bool:
        if sval not in ("SAT", "UNSAT", "UNKNOWN"):
            raise RuntimeError("invalid value")
        setattr(conf, attr, sval)
        return True
    return parse


class IncApp(Application):
    '''
    The example application implemeting incremental solving.
    '''
    program_name: str = "inc-example"
    version: str = "1.0"
    _conf: IncConfig

    def __init__(self):
        self._conf = IncConfig()

    def register_options(self, options: ApplicationOptions):
        '''
        Register program options.
        '''
        group = "Inc-Example Options"

        options.add(
            group, "imin",
            "Minimum number of steps [{}]".format(self._conf.imin),
            parse_int(self._conf, "imin", min_value=0),
            argument="<n>")

        options.add(
            group, "imax",
            "Maximum number of steps [{}]".format(self._conf.imax),
            parse_int(self._conf, "imax", min_value=0, optional=True),
            argument="<n>")

        options.add(
            group, "istop",
            "Stop criterion [{}]".format(self._conf.istop),
            parse_stop(self._conf, "istop"))

    def main(self, ctl: Control, files: Sequence[str]):
        '''
        The main function implementing incremental solving.
        '''
        if not files:
            files = ["-"]
        for file_ in files:
            ctl.load(file_)
        ctl.add("check", ["t"], "#external query(t).")

        conf = self._conf
        step = 0
        ret: Optional[SolveResult] = None

        while ((conf.imax is None or step < conf.imax) and
               (ret is None or step < conf.imin or (
                   (conf.istop == "SAT" and not ret.satisfiable) or
                   (conf.istop == "UNSAT" and not ret.unsatisfiable) or
                   (conf.istop == "UNKNOWN" and not ret.unknown)))):
            parts = []
            parts.append(("check", [Number(step)]))
            if step > 0:
                ctl.release_external(Function("query", [Number(step - 1)]))
                parts.append(("step", [Number(step)]))
            else:
                parts.append(("base", []))
            ctl.ground(parts)

            ctl.assign_external(Function("query", [Number(step)]), True)
            ret, step = cast(SolveResult, ctl.solve()), step + 1


clingo_main(IncApp(), sys.argv[1:])

Overwriting inc.py


In [10]:
!python inc.py tohE.lp tohI.lp

inc-example version 1.0
Reading from tohE.lp ...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Answer: 1
move(4,b,1) move(3,c,2) move(4,c,3) move(2,b,4) move(4,a,5) move(3,b,6) move(4,b,7) move(1,c,8) move(4,c,9) move(3,a,10) move(4,a,11) move(2,c,12) move(4,b,13) move(3,c,14) move(4,c,15)
SATISFIABLE

Models       : 1+
Calls        : 16
Time         : 0.027s (Solving: 0.01s 1st Model: 0.00s Unsat: 0.01s)
CPU Time     : 0.031s


tohE.lp:22:1-14: info: no atoms over signature occur in program:
  move/3

